#### Loading the dataset and necessary libraries

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np

# Load your dataset
data_files = {"train": "../../data_splits/train-data-split.csv", "test": "../../data_splits/test-data-split.csv"}
dataset = load_dataset("csv", data_files=data_files)

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Consumer complaint narrative', 'Product', 'Sub-product', 'Issue', 'Sub-issue'],
        num_rows: 185637
    })
    test: Dataset({
        features: ['Consumer complaint narrative', 'Product', 'Sub-product', 'Issue', 'Sub-issue'],
        num_rows: 61880
    })
})

#### Preparing the train and test set

In [4]:
product_categories = np.unique(dataset['train']['Product'])
product_mapping = {k : i for i,k in enumerate(product_categories)}

def encode_categories(batch):
    return {"labels": [product_mapping[category] for category in batch['Product']]}
    
dataset['train'] = dataset['train'].map(encode_categories , batched=True)
dataset['test'] = dataset['test'].map(encode_categories, batched=True)

Map:   0%|          | 0/185637 [00:00<?, ? examples/s]

Map:   0%|          | 0/61880 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Consumer complaint narrative', 'Product', 'Sub-product', 'Issue', 'Sub-issue', 'labels'],
        num_rows: 185637
    })
    test: Dataset({
        features: ['Consumer complaint narrative', 'Product', 'Sub-product', 'Issue', 'Sub-issue', 'labels'],
        num_rows: 61880
    })
})

#### Loading the tokenizer

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["Consumer complaint narrative"], truncation=True, padding="max_length", max_length=512)

#### Mapping into batches

In [8]:
tokenized_data = dataset.map(preprocess_function, batched=True, remove_columns=['Product', 'Sub-product','Issue', 'Sub-issue'])

Map:   0%|          | 0/185637 [00:00<?, ? examples/s]

Map:   0%|          | 0/61880 [00:00<?, ? examples/s]

In [9]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['Consumer complaint narrative', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 185637
    })
    test: Dataset({
        features: ['Consumer complaint narrative', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 61880
    })
})

In [10]:
tokenized_data = tokenized_data.remove_columns('Consumer complaint narrative')

In [11]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 185637
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 61880
    })
})

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#### Defining the metrics

In [13]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    result = {}

    metric_f1 = load_metric("f1")
    metric_precision = load_metric("precision")
    metric_recall = load_metric("recall")
    metric_acc = load_metric("accuracy")
    
    result["accuracy"] = metric_acc.compute(predictions = predictions, references = labels)["accuracy"]
    result["precision"] = metric_precision.compute(predictions = predictions, references = labels,average = 'macro')['precision']
    result["recall"] = metric_recall.compute(predictions = predictions, references = labels,average = 'macro')["recall"]
    result["f1"] = metric_f1.compute(predictions = predictions, references = labels, average = 'macro')["f1"]
    
    return result

In [16]:
id2label = {v : k for k,v in product_mapping.items()}
label2id = product_mapping

#### Loading the pre-trained model

In [17]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=len(id2label), id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
    output_dir="distil-bert-fintuned-product-cfpb-complaints",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


#### Fine-tuning the model

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.141100,0.139298,0.959502,0.878792,0.817452,0.843201


/tmp/ipykernel_8050/3516474932.py:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_f1 = load_metric("f1")
/home/kudupudi.n/.conda/envs/pytorch_env/lib/python3.10/site-packages/datasets/load.py:753: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/kudupudi.n/.conda/envs/pytorch_env/lib/python3.10/site-packages/datasets/load.py:753: FutureWarning: The repository for precision contains custom code which must be executed to correct

TrainOutput(global_step=11603, training_loss=0.16772073747206995, metrics={'train_runtime': 2404.7529, 'train_samples_per_second': 77.196, 'train_steps_per_second': 4.825, 'total_flos': 2.459216611302912e+16, 'train_loss': 0.16772073747206995, 'epoch': 1.0})

#### Getting the predictions

In [20]:
predictions = trainer.predict(tokenized_data['test'])

/home/kudupudi.n/.conda/envs/pytorch_env/lib/python3.10/site-packages/datasets/load.py:753: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/kudupudi.n/.conda/envs/pytorch_env/lib/python3.10/site-packages/datasets/load.py:753: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/precision/precision.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code

In [21]:
trainer.push_to_hub("Mahesh9/distil-bert-fintuned-product-cfpb-complaints")

CommitInfo(commit_url='https://huggingface.co/Mahesh9/distil-bert-fintuned-product-cfpb-complaints/commit/ca5c134b22d1171626f54e87bada1cd060243e89', commit_message='Mahesh9/distil-bert-fintuned-product-cfpb-complaints', commit_description='', oid='ca5c134b22d1171626f54e87bada1cd060243e89', pr_url=None, pr_revision=None, pr_num=None)

In [24]:
predicted_categories = np.argmax(predictions.predictions, axis=-1)
true_categories = [label2id[label] for label in dataset["test"]["Product"]]

#### Classification report

In [26]:
report = classification_report(true_categories, predicted_categories, target_names=product_categories)
print(report)

                             precision    recall  f1-score   support

Checking or savings account       0.94      0.93      0.93      3071
           Credit Reporting       0.98      0.98      0.98     52924
        Credit/Prepaid Card       0.80      0.87      0.83      2994
            Debt collection       0.79      0.61      0.69      2345
           Loans / Mortgage       0.89      0.70      0.78       546

                   accuracy                           0.96     61880
                  macro avg       0.88      0.82      0.84     61880
               weighted avg       0.96      0.96      0.96     61880



In [27]:
# Saving the results
import pandas as pd

true_labels_text = [id2label[label] for label in true_categories]
predicted_labels_text = [id2label[label] for label in predicted_categories]

results_df = pd.DataFrame({
    'True Labels': true_labels_text,
    'Predicted Labels': predicted_labels_text
})

original_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in tokenized_data["test"]["input_ids"]]
results_df['Complaint Narrative'] = original_texts

In [28]:
# Dumping the saved results into a csv
results_df.to_csv('distil-bert-fintuned-product-cfpb-complaints/saved_results/model_predictions_and_labels.csv', index=False)